# Ejercicio final

Hacer web scraping para obtener datos de una pagina como wikipedia y realizar un modelo para predecir el IDH de los paises leidos

In [130]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score
from sklearn import linear_model
from bs4 import BeautifulSoup
import requests, re, time
import pandas as pd 
import numpy as np 

pagina = requests.get("https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_de_Am%C3%A9rica_Latina_por_%C3%ADndice_de_desarrollo_humano")
soup = BeautifulSoup(pagina.content, "html.parser")

In [2]:
tabla = soup.find('table', attrs={'class': 'wikitable sortable'})

In [3]:
print(tabla)

<table class="wikitable sortable">
<tbody><tr>
<th>País
</th>
<th>2018
</th>
<th>2015
</th>
<th>2010
</th>
<th>2005
</th>
<th>2000
</th>
<th>1995
</th>
<th>1990
</th>
<th>Cambio
</th>
<th>Porcentaje
</th></tr>
<tr>
<td><span class="flagicon"><img alt="Bandera de Chile" class="thumbborder" data-file-height="1000" data-file-width="1500" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/7/78/Flag_of_Chile.svg/20px-Flag_of_Chile.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/7/78/Flag_of_Chile.svg/30px-Flag_of_Chile.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/7/78/Flag_of_Chile.svg/40px-Flag_of_Chile.svg.png 2x" title="Bandera de Chile" width="20"/></span><a href="/wiki/Chile" title="Chile">Chile</a>
</td>
<td>0,847
</td>
<td>0,839
</td>
<td>0,800
</td>
<td>0,788
</td>
<td>0,753
</td>
<td>0,726
</td>
<td>0,703
</td>
<td>0,144
</td>
<td>20.48%
</td></tr>
<tr>
<td><span class="flagicon"><img alt="Bandera de Argentina" class="th

In [75]:
nombreColumnas = []
for elem in tabla.find_all("th"):
    if str(elem.string)[0] == "2" or str(elem.string)[0] == "1":
        nombreColumnas.append(elem.string[:-1])
        print(elem.string[:-1])

2018
2015
2010
2005
2000
1995
1990


In [115]:
for elem in tabla.find_all('td'):
    if str(elem.string).find("Latinoamérica") == True:
        break
    if str(elem.string) and not elem.find('span') and not str(elem.string)[0] != "0":
        print(elem.string)

0,847

0,839

0,800

0,788

0,753

0,726

0,703

0,144

0,830

0,828

0,818

0,777

0,770

0,731

0,707

0,123

0,808

0,802

0,774

0,756

0,742

0,710

0,692

0,116

0,795

0,782

0,758

0,739

0,719

0,688

0,659

0,136

0,794

0,786

0,754

0,727

0,711

0,686

0,655

0,139

0,779

0,768

0,777

0,730

0,686

0,654

0,676

0,102

0,767

0,759

0,739

0,729

0,705

0,672

0,652

0,115

0,761

0,755

0,726

0,699

0,684

0,650

0,613

0,148

0,761

0,753

0,729

0,692

0,662

0,633

0,599

0,162

0,745

0,733

0,700

0,675

0,653

0,624

0,593

0,152

0,726

0,763

0,753

0,714

0,672

0,661

0,638

0,088

0,730

0,718

0,692

0,667

0,640

0,622

0,588

0,136

0,703

0,685

0,656

0,632

0,616

0,578

0,540

0,163

0,667

0,660

0,659

0,638

0,608

0,573

0,529

0,138

0,651

0,646

0,602

0,571

0,546

0,507

0,478

0,173

0,651

0,644

0,614

0,593

0,568

0,523

0,494

0,157

0,623

0,613

0,598

0,581

0,555

0,530

0,508

0,115

0,503

0,493

0,467

0,451

0,440

0,421

0,412


In [116]:
paises = []
for elem in tabla.find_all(["td", "img"]):
    #print(elem.a)
    if str(elem.string).find("America Latina y el Caribe") != -1:
        print(elem.string)
        paises.append(elem.string)
    if elem.img is not None:
        print(elem.find_next("a").string)
        paises.append(elem.find_next("a").string)

Chile
Argentina
Uruguay
Costa Rica
Cuba
Panamá
México
Brasil
Colombia
Guatemala
República Dominicana
Paraguay
Bolivia
Venezuela
El Salvador
Nicaragua
Honduras
Haití


In [118]:
paises

['Chile',
 'Argentina',
 'Uruguay',
 'Costa Rica',
 'Cuba',
 'Panamá',
 'México',
 'Brasil',
 'Colombia',
 'Guatemala',
 'República Dominicana',
 'Paraguay',
 'Bolivia',
 'Venezuela',
 'El Salvador',
 'Nicaragua',
 'Honduras',
 'Haití']

In [123]:
valores = []
for elem in tabla.find_all("td"):
    if str(elem.string).find("Latinoamérica") != -1:
        break
    if str(elem.string)[0] == "0":
        valor = float(elem.string[:-1].replace(",","."))
        if valor >=0.4:
            valores.append(valor)

In [124]:
valores = np.array(valores)
valores

array([0.847, 0.839, 0.8  , 0.788, 0.753, 0.726, 0.703, 0.83 , 0.828,
       0.818, 0.777, 0.77 , 0.731, 0.707, 0.808, 0.802, 0.774, 0.756,
       0.742, 0.71 , 0.692, 0.795, 0.782, 0.758, 0.739, 0.719, 0.688,
       0.659, 0.794, 0.786, 0.754, 0.727, 0.711, 0.686, 0.655, 0.779,
       0.768, 0.777, 0.73 , 0.686, 0.654, 0.676, 0.767, 0.759, 0.739,
       0.729, 0.705, 0.672, 0.652, 0.761, 0.755, 0.726, 0.699, 0.684,
       0.65 , 0.613, 0.761, 0.753, 0.729, 0.692, 0.662, 0.633, 0.599,
       0.745, 0.733, 0.7  , 0.675, 0.653, 0.624, 0.593, 0.726, 0.763,
       0.753, 0.714, 0.672, 0.661, 0.638, 0.73 , 0.718, 0.692, 0.667,
       0.64 , 0.622, 0.588, 0.703, 0.685, 0.656, 0.632, 0.616, 0.578,
       0.54 , 0.667, 0.66 , 0.659, 0.638, 0.608, 0.573, 0.529, 0.651,
       0.646, 0.602, 0.571, 0.546, 0.507, 0.478, 0.651, 0.644, 0.614,
       0.593, 0.568, 0.523, 0.494, 0.623, 0.613, 0.598, 0.581, 0.555,
       0.53 , 0.508, 0.503, 0.493, 0.467, 0.451, 0.44 , 0.421, 0.412])

In [125]:
valores.shape = (len(paises), len(nombreColumnas))

In [126]:
valores

array([[0.847, 0.839, 0.8  , 0.788, 0.753, 0.726, 0.703],
       [0.83 , 0.828, 0.818, 0.777, 0.77 , 0.731, 0.707],
       [0.808, 0.802, 0.774, 0.756, 0.742, 0.71 , 0.692],
       [0.795, 0.782, 0.758, 0.739, 0.719, 0.688, 0.659],
       [0.794, 0.786, 0.754, 0.727, 0.711, 0.686, 0.655],
       [0.779, 0.768, 0.777, 0.73 , 0.686, 0.654, 0.676],
       [0.767, 0.759, 0.739, 0.729, 0.705, 0.672, 0.652],
       [0.761, 0.755, 0.726, 0.699, 0.684, 0.65 , 0.613],
       [0.761, 0.753, 0.729, 0.692, 0.662, 0.633, 0.599],
       [0.745, 0.733, 0.7  , 0.675, 0.653, 0.624, 0.593],
       [0.726, 0.763, 0.753, 0.714, 0.672, 0.661, 0.638],
       [0.73 , 0.718, 0.692, 0.667, 0.64 , 0.622, 0.588],
       [0.703, 0.685, 0.656, 0.632, 0.616, 0.578, 0.54 ],
       [0.667, 0.66 , 0.659, 0.638, 0.608, 0.573, 0.529],
       [0.651, 0.646, 0.602, 0.571, 0.546, 0.507, 0.478],
       [0.651, 0.644, 0.614, 0.593, 0.568, 0.523, 0.494],
       [0.623, 0.613, 0.598, 0.581, 0.555, 0.53 , 0.508],
       [0.503,

In [127]:
df = pd.DataFrame(valores, columns=nombreColumnas)

In [128]:
df

,2018,2015,2010,2005,2000,1995,1990
0,0.847,0.839,0.800,0.788,0.753,0.726,0.703
1,0.830,0.828,0.818,0.777,0.770,0.731,0.707
2,0.808,0.802,0.774,0.756,0.742,0.710,0.692
3,0.795,0.782,0.758,0.739,0.719,0.688,0.659
4,0.794,0.786,0.754,0.727,0.711,0.686,0.655
5,0.779,0.768,0.777,0.730,0.686,0.654,0.676
6,0.767,0.759,0.739,0.729,0.705,0.672,0.652
7,0.761,0.755,0.726,0.699,0.684,0.650,0.613
8,0.761,0.753,0.729,0.692,0.662,0.633,0.599
9,0.745,0.733,0.700,0.675,0.653,0.624,0.593


In [132]:
X = df.iloc[:,1:len(df.columns)]
Y = df.iloc[:,0]

In [133]:
kf = KFold(n_splits=4)
kf.get_n_splits(X)

regr = linear_model.LinearRegression()

resultados = []

for train_index, test_index, in kf.split(X):
    X_train, X_test = X.loc[train_index,], X.loc[test_index,]
    y_train, y_test = Y[train_index], Y[test_index]
    regr.fit(X_train, y_train)
    predicciones = regr.predict(X_test)
    print("R2: ", r2_score(y_test, predicciones))
    resultados.append(r2_score(y_test, predicciones))

print("R2 medio", np.mean(resultados))

R2:  0.3125373134355033
R2:  -12.841951495660139
R2:  -0.1467599942155977
R2:  0.7502998878932683
R2 medio -2.981468572136741


In [135]:
regr = linear_model.LinearRegression()
regr.fit(X ,Y)

LinearRegression()

In [136]:
r2_score(Y, regr.predict(X))

0.9895753306029531

In [137]:
regr.coef_

array([ 1.04553409, -0.40795729,  0.12687929,  0.80778956, -0.755434  ,
        0.17201752])

In [138]:
regr.intercept_

0.014182816040394841

In [142]:
np.sum(df.iloc[:,0:6] * regr.coef_ ,axis=1)

0     0.837376
1     0.805505
2     0.788104
3     0.780499
4     0.773320
5     0.783703
6     0.757942
7     0.739497
8     0.748735
9     0.728004
10    0.726140
11    0.720440
12    0.683394
13    0.666365
14    0.629477
15    0.635720
16    0.618393
17    0.488375
dtype: float64